### Rental Data

Lets get a basic download of rentals

In [25]:
import pandas as pd
import requests
import concurrent.futures
import json
from datetime import datetime

In [1]:
URL = "https://www.realtor.com/api/v1/hulk_main_srp"

QUERYSTRING = {"client_id": "rdc-x", "schema": "vesta"}

HEADERS = {
    "authority": "www.realtor.com",
    "accept": "application/json",
    "accept-language": "en-US,en;q=0.5",
    "content-type": "application/json",
    "origin": "https://www.realtor.com",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

QUERY = """
    query ConsumerSearchMainQuery(
        $query: HomeSearchCriteria!
        $limit: Int
        $offset: Int
        $sort: [SearchAPISort]
        $sort_type: SearchSortType
        $client_data: JSON
        $bucket: SearchAPIBucket
        ) {
        home_search: home_search(
            query: $query
            sort: $sort
            limit: $limit
            offset: $offset
            sort_type: $sort_type
            client_data: $client_data
            bucket: $bucket
        ) {
            count
            total
            results {
            property_id
            list_price
            primary
            rent_to_own {
                rent
                right_to_purchase
                provider
            }
            primary_photo(https: true) {
                href
            }
            source {
                id
                agents {
                office_name
                }
                type
                spec_id
                plan_id
            }
            community {
                property_id
                permalink
                description {
                name
                }
                advertisers {
                office {
                    hours
                    phones {
                    type
                    number
                    }
                }
                builder {
                    fulfillment_id
                }
                }
            }
            products {
                brand_name
                products
            }
            listing_id
            matterport
            virtual_tours {
                href
                type
            }
            status
            permalink
            price_reduced_amount
            other_listings {
                rdc {
                listing_id
                status
                listing_key
                primary
                }
            }
            description {
                beds
                baths
                baths_full
                baths_half
                baths_1qtr
                baths_3qtr
                garage
                stories
                type
                sub_type
                lot_sqft
                sqft
                year_built
                sold_price
                sold_date
                name
            }
            location {
                street_view_url
                address {
                line
                postal_code
                state
                state_code
                city
                coordinate {
                    lat
                    lon
                }
                }
                county {
                name
                fips_code
                }
            }
            tax_record {
                public_record_id
            }
            lead_attributes {
                show_contact_an_agent
                opcity_lead_attributes {
                cashback_enabled
                flip_the_market_enabled
                }
                lead_type
                ready_connect_mortgage {
                show_contact_a_lender
                show_veterans_united
                }
            }
            open_houses {
                start_date
                end_date
                description
                methods
                time_zone
                dst
            }
            flags {
                is_coming_soon
                is_pending
                is_foreclosure
                is_contingent
                is_new_construction
                is_new_listing(days: 14)
                is_price_reduced(days: 30)
                is_plan
                is_subdivision
            }
            list_date
            last_update_date
            coming_soon_date
            photos(limit: 2, https: true) {
                href
            }
            tags
            branding {
                type
                photo
                name
            }
            }
        }
    }
"""

In [4]:
results = []

offset = 0
# total gets updated, just need a value greater than offset for first iteration
total = offset + 1

# for the purpose of this example, creating a max
max_results = 200

while offset < total:

    print(f"handling offset {offset} in a total of {total}      ", end='\r')

    payload = {
        "query": QUERY,
        "variables": {
            "query": {
                "status": ["for_rent"],
                "primary": True,
                "search_location": {"location": "Burke, VA"}
            },
            "limit": 42,
            "offset": offset,
            "sort_type": "relevant",
            "by_prop_type": ["home"]
        },
        "operationName": "ConsumerSearchMainQuery",
        "callfrom": "SRP",
        "nrQueryType": "MAIN_SRP",
        "isClient": True,
    }

    response = requests.request(
        "POST", URL, json=payload, headers=HEADERS, params=QUERYSTRING)

    if response.status_code != 200:
        raise ValueError(f"Bad status code on response: {response.status_code}")

    try:
        data = response.json()['data']['home_search']
    except:
        print("Failed to read data, something went wrong with the request")
        raise

    total = data['total']

    response_results = data['results']
    offset += len(response_results)

    results += response_results

    if max_results and offset >= max_results:
        print("\n")
        print(f"Hit max: {max_results}")
        break

print("Done!                                            ")

Done!                                            


In [8]:
results

[{'property_id': '6428776729',
  'list_price': 1200,
  'primary': True,
  'rent_to_own': None,
  'primary_photo': {'href': 'https://ap.rdcpix.com/55adbc1ffcc52ac581b48cf47e5d1e4el-m2576841375s.jpg'},
  'source': {'id': 'AVAL',
   'agents': None,
   'type': 'unit_rental',
   'spec_id': None,
   'plan_id': None},
  'community': None,
  'products': None,
  'listing_id': '2960744790',
  'matterport': False,
  'virtual_tours': None,
  'status': 'for_rent',
  'permalink': '9814-Portside-Dr_Burke_VA_22015_M64287-76729',
  'price_reduced_amount': None,
  'other_listings': {'rdc': [{'listing_id': '2960744790',
     'status': 'for_rent',
     'listing_key': None,
     'primary': True},
    {'listing_id': '2941873491',
     'status': 'sold',
     'listing_key': None,
     'primary': None},
    {'listing_id': '528285170',
     'status': 'off_market',
     'listing_key': None,
     'primary': None}]},
  'description': {'beds': 1,
   'baths': 1,
   'baths_full': 1,
   'baths_half': None,
   'baths_1

Seems we have listings with blank listing_date that is screwing up concats later.

In [28]:
for result in results:
    print(result.get('list_date'))

2023-10-20T19:06:59Z
2023-11-12T21:20:01Z
2023-10-09T19:18:44Z
2023-10-28T14:43:58Z
2023-10-30T04:15:27Z
2023-11-09T18:31:26Z
2023-11-10T05:15:07Z
2023-10-05T04:02:25Z
2023-09-05T17:09:37Z
None
None
2023-11-03T01:04:27Z
2023-11-10T17:09:00Z
2023-09-28T16:13:19Z
2023-10-13T04:16:40Z
2023-10-27T19:17:28Z
None
2023-11-11T02:12:00Z
2023-10-05T21:19:13Z
2023-11-02T20:19:15Z
2023-11-03T20:11:13Z
2023-09-06T20:06:33Z
2023-10-24T16:29:48Z
2023-11-10T16:25:04Z
2023-11-03T14:26:10Z
2023-08-17T04:15:24Z
2023-11-07T05:12:48Z
2023-08-03T15:20:53Z
2023-11-05T04:14:23Z
2023-11-03T21:41:49Z
2023-10-30T13:59:02Z
None
None
2023-11-13T03:43:05Z
2023-11-02T19:32:09Z
2023-09-24T04:12:42Z
2023-10-23T22:12:12Z
2023-09-29T19:13:33Z
2023-08-03T03:11:11Z
2023-11-14T11:06:31Z


In [33]:
filtered_results = [result for result in results if result.get('list_date') is None]
filtered_results

[{'property_id': '9237732046',
  'list_price': 2457,
  'primary': True,
  'rent_to_own': None,
  'primary_photo': {'href': 'https://ap.rdcpix.com/09b387cf535adfd832140e9927776540l-m3280094454s.jpg'},
  'source': {'id': 'RNBT',
   'agents': None,
   'type': 'unit_rental',
   'spec_id': None,
   'plan_id': None},
  'community': None,
  'products': None,
  'listing_id': '2953139299',
  'matterport': False,
  'virtual_tours': None,
  'status': 'for_rent',
  'permalink': '5812-Chase-Commons-Ct-1-09204_Burke_VA_22015_M92377-32046',
  'price_reduced_amount': 201,
  'other_listings': {'rdc': [{'listing_id': '2953139299',
     'status': 'for_rent',
     'listing_key': None,
     'primary': True}]},
  'description': {'beds': 2,
   'baths': 2,
   'baths_full': 2,
   'baths_half': None,
   'baths_1qtr': None,
   'baths_3qtr': None,
   'garage': None,
   'stories': None,
   'type': 'apartment',
   'sub_type': 'condo',
   'lot_sqft': None,
   'sqft': 1097,
   'year_built': None,
   'sold_price': Non

In [38]:
print(filtered_results[0].get('list_date') or datetime.now())

2023-11-14 12:10:55.170768


### Parse Code

In [31]:
def parse_flags(flags):
    # flags example value
    # {'is_coming_soon': None, 'is_new_listing': False, 'is_price_reduced': None, 'is_foreclosure': None, 'is_new_construction': None, 'is_pending': True, 'is_contingent': None}

    status = []
    if flags.get('is_coming_soon') is True:
        status.append("coming soon")
    if flags.get('is_new_listing') is True:
        status.append("new listing")
    if flags.get('is_price_reduced') is True:
        status.append("price reduced")
    if flags.get('is_foreclosure') is True:
        status.append("foreclosure")
    if flags.get('is_new_construction') is True:
        status.append("new construction")
    if flags.get('is_pending') is True:
        status.append("pending")
    if flags.get('is_contingent') is True:
        status.append("contingent")

    return ", ".join(status)

In [39]:
df = pd.DataFrame()

for result in results:

    if result['location']['address'].get('coordinate'):
        lat = result['location']['address'].get('coordinate', {}).get('lat')
        lon = result['location']['address'].get('coordinate', {}).get('lon')
    else:
        lat, lon = None, None

    interdf = pd.DataFrame([{
            'id': result.get('property_id'),
            'list date': pd.to_datetime(result.get('list_date') or datetime.now()),
            'status': result['status'],
            'flags': parse_flags(result['flags']),
            'home type': result['description']['type'],
            'year built': result.get('description').get('year_built', ""),
            'price': result['list_price'],
            'hoa fee': result.get('hoa', {}).get('fee', None),
            'beds': result['description']['beds'],
            'baths': result['description']['baths'],
            'interior sqft': result['description']['sqft'],
            'lot sqft': result['description']['lot_sqft'],
            'address': f"{result['location']['address']['line']} {result['location']['address']['city']}, {result['location']['address']['state_code']} {result['location']['address']['postal_code']}",
            'addresss line': result['location']['address']['line'],
            'city': result['location']['address']['city'],
            'state': result['location']['address']['state_code'],
            'zipcode': result['location']['address']['postal_code'],
            'latitude': lat,
            'longitude': lon,
            'url': f"https://www.realtor.com/realestateandhomes-detail/{result['permalink']}",
        }])
    df = pd.concat([df, interdf])

print(df.shape)
df.head()

(40, 20)


/var/folders/2l/88ghk2rn2572_7lbmkzzf9r00000gn/T/ipykernel_2663/4075686559.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, interdf])


,id,list date,status,flags,home type,year built,price,hoa fee,beds,baths,interior sqft,lot sqft,address,addresss line,city,state,zipcode,latitude,longitude,url
0,6428776729,2023-10-20 19:06:59+00:00,for_rent,,single_family,None,1200,None,1,1,None,None,"9814 Portside Dr Burke, VA 22015",9814 Portside Dr,Burke,VA,22015,38.764381,-77.285435,https://www.realtor.com/realestateandhomes-det...
0,6368439317,2023-11-12 21:20:01+00:00,for_rent,new listing,single_family,1985,3800,None,4,4,3276,9230,"9296 Mainsail Dr Burke, VA 22015",9296 Mainsail Dr,Burke,VA,22015,38.771223,-77.265583,https://www.realtor.com/realestateandhomes-det...
0,6351673495,2023-10-09 19:18:44+00:00,for_rent,contingent,single_family,1983,3295,None,4,4,3024,11761,"9469 Lapstrake Ln Burke, VA 22015",9469 Lapstrake Ln,Burke,VA,22015,38.768023,-77.271601,https://www.realtor.com/realestateandhomes-det...
0,6588909004,2023-10-28 14:43:58+00:00,for_rent,,single_family,1972,3400,None,4,3,3029,9525,"9089 Andromeda Dr Burke, VA 22015",9089 Andromeda Dr,Burke,VA,22015,38.783805,-77.260462,https://www.realtor.com/realestateandhomes-det...
0,6213097199,2023-10-30 04:15:27+00:00,for_rent,,condos,1974,2300,None,3,3,1512,None,"6459 Fenestra Ct Unit 51C Burke, VA 22015",6459 Fenestra Ct Unit 51C,Burke,VA,22015,38.777638,-77.260706,https://www.realtor.com/realestateandhomes-det...
